**What I want to do:** Get suburb information from microburbs (fantastic site btw) and understand which place would be ideal for us to move next.

Microburbs is a fantastic site that has aggregated useful information from several government sources as well as made intelligent derivations (pretty much what I would have done with the raw information - only more detailed and smarter!). However, the only way to see information in microburbs is through a map hover feature which makes it really hard to see and compare information from  all the surrounding suburbs of my suburb of interest. So let me get information of all the suburbs and then simply compare them side by side.

## 1. Admin stuff

In [1]:
import bs4
from bs4 import BeautifulSoup
from urllib2 import urlopen
import pandas as pd
import re
import numpy as np
from time import sleep
%matplotlib inline
import pylab as plt
import os
import warnings
import time

## 2. Source suburb html

I hovered over the inner west region of Sydney in microburbs.com.au and got the areas-list element upon inspection. Will go ahead and store this in a variable and extract info from the html.

In [2]:
baseid = 'https://www.microburbs.com.au'

In [2]:
# open the file and read as a single buffer. Close the file after reading successfully
fd = open('dict_burb_html', 'r')
cache = fd.read()
fd.close()

In [3]:
# this line of code splits ther html I've collected into burb locations. What I want to do is mine the html and collect burbids and burblinks
dict_burb_html = cache.replace('\n','').split(',')

## 3. Mine html and get the burbids/burblinks

In [4]:
# write a function for this:
def get_burbid_and_burblink(location, html_codedump, baseid = 'https://www.microburbs.com.au'):
    soup = BeautifulSoup(html_codedump, "lxml")
    burbid=[]; burblink = []
    for a in soup.find_all('a', href=True):
        burbid.append(a.string.string.encode('utf-8'))
        burblink.append(a['href'])
    df_temp = pd.DataFrame({'location':[location for x in burbid],'burbid':burbid,'burblink':burblink})
    df_temp['burblink'] = df_temp['burblink'].apply(lambda x: baseid+x)
    return df_temp

In [5]:
df_burbs = pd.DataFrame()
for burb in dict_burb_html:
    df_burbs = pd.concat((df_burbs, get_burbid_and_burblink(burb.split('|')[0],burb.split('|')[1])))

In [6]:
df_burbs.shape

(148, 3)

In [7]:
df_burbs.burbid = pd.to_numeric(df_burbs.burbid, errors='coerce').fillna(0).astype(np.int64)

In [8]:
# change burb dataframe index
df_burbs.set_index('burbid',inplace=True)

In [9]:
pd.options.display.max_colwidth = 100
df_burbs.tail()

,burblink,location
burbid,,
1141559,https://www.microburbs.com.au/NSW/Sydney/Mosman-Municipality/Mosman/1141559,Mosman near wharf
1141560,https://www.microburbs.com.au/NSW/Sydney/Mosman-Municipality/Mosman/1141560,Mosman near wharf
1141549,https://www.microburbs.com.au/NSW/Sydney/Mosman-Municipality/Mosman/1141549,Mosman near wharf
1141564,https://www.microburbs.com.au/NSW/Sydney/Mosman-Municipality/Mosman/1141564,Mosman near wharf
1141554,https://www.microburbs.com.au/NSW/Sydney/Mosman-Municipality/Mosman/1141554,Mosman near wharf


## 4. Mine html elements to get scores - deprecated. Use Method 2

In [ ]:
blah. Placeholder to stop code execution flow. Go to method 2.

In [13]:
def get_burb_scores(burb_full_link):
    # parse the contents of the link and store inside the soup
    html = urlopen(burb_full_link).read()  
    soup = BeautifulSoup(html, 'lxml')
    
    # this class has all the tabular info
    tab = soup.findAll('div', attrs = {'class' : 'col-sm-6 col-lg-5'})
    
    scores=[]
    for targetElements in soup.findAll('div', attrs = {'class' : 'col-sm-6 col-lg-5'}):
        for individual_score in targetElements.findAll('span', attrs = {'class' : 'human-score-value'}):
            scores.append(int(individual_score.text.strip().encode('utf-8')))
    
    return scores

In [48]:
burb_scores = {}
for rowid,row in df_test.iterrows():
    scores_before_check = get_burb_scores(row['burblink'])
    if len(scores_before_check)!=9:
        burb_scores[rowid]=np.repeat(99,9).tolist()
    else:
        burb_scores[rowid]=scores_before_check
    if rowid%10==0:
        print (rowid,'burbs completed...')
    time.sleep(5)

('1141549', 'completed. sleeping for 10 seconds...')
('1141564', 'completed. sleeping for 10 seconds...')
('1141554', 'completed. sleeping for 10 seconds...')


{'1141549': [7, 10, 10, 8, 9, 9, 9, 9, 10],
 '1141554': [7, 10, 10, 9, 9, 8, 10, 9, 10],
 '1141564': [99, 99, 99, 99, 99, 99, 99, 99, 99]}

In [56]:
df_test1 = pd.DataFrame.from_dict(burb_scores, orient='index')
df_test1.columns = ['Hip','Family','Affluence','Safety','Lifestyle','Convenience','Tranquility','Internet','Community']
df_test1

,Hip,Family,Affluence,Safety,Lifestyle,Convenience,Tranquility,Internet,Community
1141564,99,99,99,99,99,99,99,99,99
1141549,7,10,10,8,9,9,9,9,10
1141554,7,10,10,9,9,8,10,9,10


In [58]:
pd.merge(left=df_burbs, right=df_test1, left_index=True, right_index=True)

,burblink,location,Hip,Family,Affluence,Safety,Lifestyle,Convenience,Tranquility,Internet,Community
1141549,https://www.microburbs.com.au/NSW/Sydney/Mosman-Municipality/Mosman/1141549,Mosman near wharf,7,10,10,8,9,9,9,9,10
1141554,https://www.microburbs.com.au/NSW/Sydney/Mosman-Municipality/Mosman/1141554,Mosman near wharf,7,10,10,9,9,8,10,9,10
1141564,https://www.microburbs.com.au/NSW/Sydney/Mosman-Municipality/Mosman/1141564,Mosman near wharf,99,99,99,99,99,99,99,99,99


My main concern with this method is the 99 I need to enter. Even if only 1 value is not populated, the function is not smart enough to map the values to the correct column name (because of the hardcoding). So I need to forego the entire burb, which is undesirable. Lets see if there is a better method.

## 5. Explore html element for scores - Method 2

In [147]:
burbid=1141564
burb_full_link = 'https://www.microburbs.com.au/NSW/Sydney/Mosman-Municipality/Mosman/1141564'

In [64]:
html = urlopen(burb_full_link).read()  
soup = BeautifulSoup(html, 'lxml')
tab = soup.findAll('div', attrs = {'class' : 'col-sm-6 col-lg-5'})

In [145]:
dict={}
for targetElements in soup.findAll('div', attrs = {'class' : 'col-sm-6 col-lg-5'}):
    for individual_score in targetElements.findAll('span', attrs = {'class' : 'human-score-value'}):
        dict[individual_score.parent.find('span', attrs = {'class' : 'title'}).text.strip().encode('utf-8')]=int(individual_score.text.strip().encode('utf-8'))
dict

{'Affluence Score': 10, 'Communications Score': 4, 'Safety Score': 10}

In [149]:
a = pd.DataFrame(dict, index=[burbid])
a

,Affluence Score,Communications Score,Safety Score
1141564,10,4,10


In [150]:
burbid=1141554
burb_full_link = 'https://www.microburbs.com.au/NSW/Sydney/Mosman-Municipality/Mosman/1141554'
html = urlopen(burb_full_link).read()  
soup = BeautifulSoup(html, 'lxml')
dict={}
for targetElements in soup.findAll('div', attrs = {'class' : 'col-sm-6 col-lg-5'}):
    for individual_score in targetElements.findAll('span', attrs = {'class' : 'human-score-value'}):
        dict[individual_score.parent.find('span', attrs = {'class' : 'title'}).text.strip().encode('utf-8')]=int(individual_score.text.strip().encode('utf-8'))
dict

{'Affluence Score': 10,
 'Communications Score': 9,
 'Community Score': 10,
 'Convenience Score': 8,
 'Family Score': 10,
 'Hip Score': 7,
 'Lifestyle Score': 9,
 'Safety Score': 9,
 'Tranquillity Score': 10}

In [151]:
b = pd.DataFrame(dict, index=[burbid])
b

,Affluence Score,Communications Score,Community Score,Convenience Score,Family Score,Hip Score,Lifestyle Score,Safety Score,Tranquillity Score
1141554,10,9,10,8,10,7,9,9,10


In [152]:
pd.concat([a,b])

,Affluence Score,Communications Score,Community Score,Convenience Score,Family Score,Hip Score,Lifestyle Score,Safety Score,Tranquillity Score
1141564,10,4,NaN,NaN,NaN,NaN,NaN,10,NaN
1141554,10,9,10.0,8.0,10.0,7.0,9.0,9,10.0


Pandas is legend!

### 5.1 Tying it all into a function

In [10]:
def get_burb_scores(burblink,burbid):
    html = urlopen(burblink).read()  
    soup = BeautifulSoup(html, 'lxml')
    dict={}
    for targetElements in soup.findAll('div', attrs = {'class' : 'col-sm-6 col-lg-5'}):
        for individual_score in targetElements.findAll('span', attrs = {'class' : 'human-score-value'}):
            dict[individual_score.parent.find('span', attrs = {'class' : 'title'}).text.strip().encode('utf-8')]=int(individual_score.text.strip().encode('utf-8'))
    return pd.DataFrame(dict, index=[burbid])

In [48]:
df_scores = pd.DataFrame()
#***************************************************************************
#Commenting out the next few lines because this was a one time activity
# I'll read from the csv for all subsequent runs
# counter = 0
# for rowid,row in df_burbs.iterrows():
#     df_scores = pd.concat([df_scores,get_burb_scores(row['burblink'], rowid)])
#     counter+=1
#     if counter%10 == 0:
#         print (counter,' records completed...')
#     time.sleep(10)
# df_scores.to_csv('burb_scores',index=True,header=True)
df_scores = pd.read_csv('burb_scores', index_col=0)

In [49]:
df_scores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148 entries, 1139512 to 1141554
Data columns (total 9 columns):
Affluence Score         148 non-null int64
Communications Score    148 non-null int64
Community Score         147 non-null float64
Convenience Score       147 non-null float64
Family Score            144 non-null float64
Hip Score               147 non-null float64
Lifestyle Score         147 non-null float64
Safety Score            148 non-null int64
Tranquillity Score      147 non-null float64
dtypes: float64(6), int64(3)
memory usage: 11.6 KB


In [50]:
df_scores.head()

,Affluence Score,Communications Score,Community Score,Convenience Score,Family Score,Hip Score,Lifestyle Score,Safety Score,Tranquillity Score
1139512,9,9,9.0,9.0,8.0,9.0,9.0,3,7.0
1139514,9,9,9.0,9.0,9.0,8.0,9.0,3,7.0
1139501,10,9,9.0,9.0,10.0,8.0,9.0,5,8.0
1139502,10,9,9.0,9.0,9.0,8.0,9.0,3,8.0
1139516,10,9,10.0,9.0,10.0,8.0,9.0,3,8.0


In [54]:
df_combined = pd.merge(left=df_burbs,right=df_scores,how='inner',left_index=True,right_index=True)
df_combined[df_combined['Safety Score']>7].location.value_counts()

Mosman near wharf              10
Macmohans poimnt near wharf     3
Name: location, dtype: int64

## 6. Explore html element for prices

I'm happy with the dict approach. I should be able to map local prices as well as ethcicity by the same token.

In [11]:
burbid=1141564
burb_full_link = 'https://www.microburbs.com.au/NSW/Sydney/Mosman-Municipality/Mosman/1141564'

In [12]:
html = urlopen(burb_full_link).read()  
soup = BeautifulSoup(html, 'lxml')
tab = soup.findAll('div', attrs = {'class' : 'col-sm-6 col-lg-5'})

tab doesnt have the actual prices. I'm only able to find the link to local prices in the attribute: data-target="#localprices-score-collapse"

In [13]:
tab[0].findAll('div',attrs={'data-target':'#localprices-score-collapse'})

[<div aria-expanded="false" class="panel-heading collapsed" data-target="#localprices-score-collapse" data-toggle="collapse">\n<div class="panel-control"></div>\n<h3 class="panel-title">\n<i class="fa fa-line-chart"></i>\xa0\r\n    \r\n\r\n    \r\n        <span class="title">Local Prices</span>\n</h3>\n</div>]

In [13]:
from selenium import webdriver
driver = webdriver.Chrome()
driver.get(burb_full_link)

In [15]:
html2=driver.page_source
wait = WebDriverWait(driver, 8)
#soup2 = BeautifulSoup(html2, 'html.parser')
#tab2 = soup2.findAll('div', attrs = {'class' : 'col-sm-6 col-lg-5'})

In [24]:
blah = pd.read_html(html2, flavor='bs4')

In [25]:
type(blah)

list

In [40]:
blah[8]

,Unnamed: 0,This microburb,Mosman LGA
0,Languages spoken,NaN,English 88% Italian 1% 18 more German 1% French 1% Mandarin 1% Cantonese 1% Japanes...
1,Ancestry,NaN,English 40% Australian 28% 18 more Irish 13% Scottish 11% German 4% Italian 3% Chin...


In [25]:
#

In [41]:
driver.close()

WebDriverException: Message: no such session
  (Driver info: chromedriver=2.29.461585 (0be2cd95f834e9ee7c46bcc7cf405b483f5ae83b),platform=Mac OS X 10.12.5 x86_64)
